In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.4 MB/s eta 0:00:00


In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import BertModel, BertTokenizer
from torch.utils.data import TensorDataset, Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

## Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
data_dir = '/content/drive/MyDrive/data/ethic'
train = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t', header=None)
test = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t', header=None)

In [5]:
print(len(train))
train = train.dropna()
print(len(train))

363154
363153


In [6]:
train_sen = list(train[0])
train_label = list(train[1])
train_score = list(train[2])

test_sen = list(test[0])
test_label = list(test[1])
test_score = list(test[2])

In [7]:
model_name = 'kykim/bert-kor-base'

In [8]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [9]:
max_len = 0
for sentence in train_sen:
  tokens = tokenizer.tokenize(sentence)
  length = len(tokens)
  if length > max_len:
    max_len = length

max_len

99

In [10]:
train_sen, valid_sen, train_label, valid_label, train_score, valid_score = train_test_split(train_sen, train_label, train_score, test_size=0.1, random_state=42)

In [11]:
len(train_sen), len(valid_sen)

(326837, 36316)

## DataLoader

In [12]:
class TextDataset(Dataset):
  def __init__(self, sen, label, score):
    self.text = sen
    self.label = label
    self.score = score

  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):
    text = self.text[idx]
    label = self.label[idx]
    score = self.score[idx]

    return text, label, score

In [13]:
train_dataset = TextDataset(train_sen, train_label, train_score)
valid_dataset = TextDataset(valid_sen, valid_label, valid_score)
test_dataset = TextDataset(test_sen, test_label, test_score)

In [14]:
batch = 64

train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False)

## Model

In [15]:
class Classifier(nn.Module):
  def __init__(self, model_name, hidden_dim, output_dim, dropout=0.2, device='cuda'):
    super(Classifier, self).__init__()
    self.bert = BertModel.from_pretrained(model_name)
    self.fc1 = nn.Linear(self.bert.config.hidden_size, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, output_dim)

    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    self.dropout = nn.Dropout(dropout)

    self.device = device

  def forward(self, x):
    y = self.sigmoid(self.bert(**x).pooler_output)
    y = self.dropout(y)

    y = self.sigmoid(self.fc1(y))
    y = self.dropout(y)

    y = self.sigmoid(self.fc2(y))

    return y

## Train

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [17]:
model = Classifier(model_name, 200, 1).to(device)
lr = 1e-5

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

num_epochs = 15
early_stop = 3

In [18]:
train_label[0]

True

In [18]:
from tqdm import tqdm
from time import time

In [19]:
torch.cuda.empty_cache()

In [22]:
min_loss = np.inf
stop_count = 0
for epoch in range(1, num_epochs+1):
  start = time()
  train_loss = 0.0
  valid_loss = 0.0
  model.train()
  pbar = tqdm(train_loader, desc=f'Epoch {epoch}')
  for idx, batch in enumerate(pbar):
    text, label, score = batch
    label = torch.tensor(label, dtype=torch.float).to(device)
    # score = torch.tensor(score, dtype=torch.float).to(device)
    encoding = tokenizer(text, max_length=max_len, padding='max_length', truncation=True, return_tensors='pt').to(device)

    output = model(encoding)

    loss = criterion(output.squeeze(), label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    pbar.set_postfix(loss = loss.item())

    train_loss += loss.item()

  train_loss /= idx

  with torch.no_grad():
    model.eval()
    pbar = tqdm(valid_loader, desc=f'Epoch {epoch}')

    for idx, batch in enumerate(pbar):
      text, label, score = batch
      label = torch.tensor(label, dtype=torch.float).to(device)
      # score = torch.tensor(score, dtype=torch.float).to(device)
      encoding = tokenizer(text, max_length=max_len, padding='max_length', truncation=True, return_tensors='pt').to(device)

      output = model(encoding)
      loss = criterion(output.squeeze(), label)

      pbar.set_postfix(loss = loss.item())

      valid_loss += loss.item()

    valid_loss /= idx
    if valid_loss <= min_loss:
      stop_count = 0
      min_loss = valid_loss
      best_epoch = epoch
      torch.save(model.state_dict(), os.path.join(data_dir, 'best_model.pt'))
    else:
      stop_count += 1
      if stop_count == early_stop:
        print("Early stop!")
        print(f'Best epoch : {best_epoch}')
        break

  print(f'Epoch: {epoch} | Elapsed time: {time()-start} | Training Loss: {train_loss:.3f} | Valid Loss: {valid_loss:.3f}')

Epoch 1:   0%|          | 0/5107 [00:00<?, ?it/s]<ipython-input-22-edbf6f8f018d>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.float).to(device)
Epoch 1:   0%|          | 0/568 [00:00<?, ?it/s]<ipython-input-22-edbf6f8f018d>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.float).to(device)
Epoch 1: 100%|██████████| 568/568 [03:25<00:00,  2.76it/s, loss=0.182]


Epoch: 1 | Elapsed time: 5165.651659727097 | Training Loss: 0.402 | Valid Loss: 0.347


Epoch 2: 100%|██████████| 568/568 [03:26<00:00,  2.76it/s, loss=0.165]


Epoch: 2 | Elapsed time: 5175.261150598526 | Training Loss: 0.326 | Valid Loss: 0.334


Epoch 3: 100%|██████████| 568/568 [03:25<00:00,  2.76it/s, loss=0.125]


Epoch: 3 | Elapsed time: 5168.65065741539 | Training Loss: 0.288 | Valid Loss: 0.343


Epoch 4: 100%|██████████| 568/568 [03:25<00:00,  2.76it/s, loss=0.13]


Epoch: 4 | Elapsed time: 5165.114332675934 | Training Loss: 0.251 | Valid Loss: 0.360


Epoch 5: 100%|██████████| 568/568 [03:25<00:00,  2.76it/s, loss=0.074]

Early stop!
Best epoch : 2


## Evaluate

In [47]:
pbar = tqdm(test_loader)
precision = 0.0
recall = 0.0
accuracy = 0.0

test_loss = 0.0

model.load_state_dict(torch.load(os.path.join(data_dir, 'best_model.pt')))

with torch.no_grad():
  model.eval()
  for idx, batch in enumerate(pbar):
    text, label, score = batch
    label = torch.tensor(label, dtype=torch.float).to(device)
    score = torch.tensor(score, dtype=torch.float).to(device)
    encoding = tokenizer(text, max_length=max_len, padding='max_length', truncation=True, return_tensors='pt').to(device)

    output = model(encoding)

    y = torch.where(output.squeeze()<0.5, 0, 1)

    accuracy += (torch.eq(y, label).sum().item()/len(label))
    precision += (torch.logical_and(y, label).sum().item()/y.sum().item())
    recall += (torch.logical_and(y, label).sum().item()/label.sum().item())

  precision /= idx
  recall /= idx
  accuracy /= idx
  f1_score = 2*(precision*recall)/(precision+recall)

print(f'Precision : {precision:.3f} | Recall : {recall:.3f} | Accuracy : {accuracy:.3f} | F1 score : {f1_score:.3f}')

  0%|          | 0/707 [00:00<?, ?it/s]<ipython-input-47-3f5df0428298>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.float).to(device)
<ipython-input-47-3f5df0428298>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  score = torch.tensor(score, dtype=torch.float).to(device)
100%|██████████| 707/707 [04:03<00:00,  2.90it/s]

Precision : 0.864 | Recall : 0.892 | Accuracy : 0.863 | F1 score : 0.878
